# **Análisis de Resultados Saber 11 - Departamento de Caldas**

Este notebook implementa el proceso de selección, limpieza, alistamiento y análisis exploratorio de los datos de las pruebas Saber 11 para el departamento de Caldas, como parte del Proyecto 1 del curso *Analítica Computacional para la Toma de Decisiones*. El producto final está orientado al **Ministerio de Educación** como usuario final, y busca responder tres preguntas de negocio relacionadas con equidad socioeconómica, desempeño territorial y brechas de género.

## Tarea 2 - Selección, limpieza y alistamiento de datos

Los datos provienen del portal de [Datos Abiertos de Colombia](https://www.datos.gov.co/Educaci-n/Resultados-nicos-Saber-11/kgxf-xxbe), actualizados a abril de 2024. Dado que el conjunto original contiene más de 7 millones de filas y 50 columnas (~3.4 GB), se emplearon **AWS Glue** y **AWS Athena** para extraer únicamente el subconjunto correspondiente al departamento de Caldas (≈87,000 filas) con las 26 columnas relevantes para las preguntas de negocio. El subconjunto extraído se carga en Python en el archivo **csv_reader.py**, donde se realizan la limpieza de comillas, el casteo de tipos de datos y la gestión de valores faltantes.

## Tarea 3 - Exploración y análisis de datos

Con los datos limpios y correctamente tipados, se realiza un análisis de datos faltantes antes de continuar con el análisis exploratorio orientado a responder las tres preguntas de negocio en este archivo. Esto incluye estadísticas descriptivas, histogramas, diagramas de caja, diagramas de dispersión y mapas de calor sobre las variables de puntaje, estrato, educación de los padres, municipio, naturaleza del colegio, zona y género.

---

Daniel Benavides - 202220428 - <d.benavidess@uniandes.edu.co>

Juanita Cortés - 202222129 - <jv.cortesv1@uniandes.edu.co>

Andrés Felipe Herrera - 

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import altair as alt
from prettytable import PrettyTable

from matplotlib import font_manager
plt.rcParams['font.family'] = 'Arial'

In [2]:
import warnings
warnings.filterwarnings('ignore')      

## 1. Data loading

In [3]:
df = pd.read_csv('caldas_saber11.csv')
df = df.copy()
df.columns

Index(['periodo', 'cole_mcpio_ubicacion', 'cole_depto_ubicacion',
       'punt_matematicas', 'punt_lectura_critica', 'punt_global',
       'fami_estratovivienda', 'fami_educacionmadre', 'fami_educacionpadre',
       'fami_personashogar', 'fami_cuartoshogar', 'fami_tienecomputador',
       'fami_tieneinternet', 'fami_tieneautomovil', 'fami_tienelavadora',
       'punt_c_naturales', 'punt_sociales_ciudadanas', 'punt_ingles',
       'cole_naturaleza', 'cole_area_ubicacion', 'cole_bilingue',
       'estu_genero', 'desemp_ingles'],
      dtype='object')

In [4]:
df.head(1)

,periodo,cole_mcpio_ubicacion,cole_depto_ubicacion,punt_matematicas,punt_lectura_critica,punt_global,fami_estratovivienda,fami_educacionmadre,fami_educacionpadre,fami_personashogar,...,fami_tieneautomovil,fami_tienelavadora,punt_c_naturales,punt_sociales_ciudadanas,punt_ingles,cole_naturaleza,cole_area_ubicacion,cole_bilingue,estu_genero,desemp_ingles
0,20224,MANIZALES,CALDAS,43.0,48.0,203.0,NaN,Primaria incompleta,No Aplica,NaN,...,No,Si,37.0,38.0,31.0,OFICIAL,URBANO,NaN,M,A-


In [5]:
categorical_columns = df.select_dtypes(include=['object', 'string']).columns.tolist()
print("Categorical Columns:", categorical_columns)

Categorical Columns: ['cole_mcpio_ubicacion', 'cole_depto_ubicacion', 'fami_estratovivienda', 'fami_educacionmadre', 'fami_educacionpadre', 'fami_personashogar', 'fami_cuartoshogar', 'fami_tienecomputador', 'fami_tieneinternet', 'fami_tieneautomovil', 'fami_tienelavadora', 'cole_naturaleza', 'cole_area_ubicacion', 'cole_bilingue', 'estu_genero', 'desemp_ingles']


In [6]:
numerical_columns = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
print("Numerical Columns:", numerical_columns)

Numerical Columns: ['periodo', 'punt_matematicas', 'punt_lectura_critica', 'punt_global', 'punt_c_naturales', 'punt_sociales_ciudadanas', 'punt_ingles']


### 1.1 Data cleaning and transformation

In this section, we will look through every single categorical variable in order to identify the nature of the variable, meaning we will look at its unique values and see if these entries are aligned with what the variable is referring to. If this is not the case, we will clean the columns individually so that only valid values are present in the column, and any other values will either be deleted or transferred to another column in order to facilitate the missing data analysis in **Section 2**.

#### 1.1.1 fami_estratovivienda

In [7]:
fami_estratovivienda = df['fami_estratovivienda'].unique()
print("Unique values in 'fami_estratovivienda':", fami_estratovivienda)

Unique values in 'fami_estratovivienda': [nan 'Estrato 2' 'Estrato 3' 'Estrato 1' 'Estrato 4' 'Estrato 6'
 'Estrato 5' 'Sin Estrato' 'Primaria incompleta' 'Primaria completa'
 'Ninguno' 'Educación profesional completa' 'No sabe'
 'Técnica o tecnológica completa' 'Secundaria (Bachillerato) completa'
 'Secundaria (Bachillerato) incompleta' 'Técnica o tecnológica incompleta'
 'Postgrado']


In [8]:
# Delete rows that do not have valid estrato values
valid_estratos = ['Estrato 1', 'Estrato 2', 'Estrato 3', 'Estrato 4', 'Estrato 5', 'Estrato 6']
df = df[df['fami_estratovivienda'].isin(valid_estratos)]

df['fami_estratovivienda'].unique()

array(['Estrato 2', 'Estrato 3', 'Estrato 1', 'Estrato 4', 'Estrato 6',
       'Estrato 5'], dtype=object)

#### 1.1.2 fami_educacionmadre

In [9]:
fami_educacionmadre = df['fami_educacionmadre'].unique()
print("Unique values in 'educación madre':", fami_educacionmadre)

Unique values in 'educación madre': ['Primaria incompleta' 'Secundaria (Bachillerato) completa'
 'Primaria completa' 'Ninguno' 'Secundaria (Bachillerato) incompleta'
 'No sabe' 'Postgrado' 'Técnica o tecnológica completa'
 'Educación profesional completa' 'Técnica o tecnológica incompleta' nan
 'Educación profesional incompleta' 'No Aplica']


#### 1.1.3 fami_educacionpadre

In [10]:
fami_educacionpadre = df['fami_educacionpadre'].unique()
print("Unique values in 'educación padre':", fami_educacionpadre)

Unique values in 'educación padre': ['Primaria incompleta' nan 'No sabe'
 'Secundaria (Bachillerato) incompleta' 'Ninguno' 'Primaria completa'
 'Secundaria (Bachillerato) completa' 'Educación profesional completa'
 'No Aplica' 'Técnica o tecnológica completa' 'Postgrado'
 'Educación profesional incompleta' 'Técnica o tecnológica incompleta']


#### 1.1.4 fami_personas_hogar

In [11]:
fami_personashogar = df['fami_personashogar'].unique()
print("Unique values in 'fami_personashogar':", fami_personashogar)

Unique values in 'fami_personashogar': ['3 a 4' '1 a 2' '5 a 6' '9 o más' nan '7 a 8' 'Seis' 'Cuatro' 'Siete'
 'Cinco' 'Dos' 'Tres' 'Ocho' 'Una' 'Nueve' 'Doce o más' 'Diez' 'Once']


In [12]:
df['fami_personashogar'] = df['fami_personashogar'].replace({
    '1 a 2': '2',
    '3 a 4': '3',
    '5 a 6': '5',
    '7 a 8': '7',
    '9 o más': '9',
    'Uno': '1',
    'Dos': '2',
    'Tres': '3',
    'Cuatro': '4',
    'Cinco': '5',
    'Seis': '6',
    'Siete': '7',
    'Ocho': '8',
    'Nueve': '9',
    'Diez': '10',
    'Once': '11',
    'Doce o más': '12'
})

# Convert to numeric integer, coercing errors to NaN
df['fami_personashogar'] = pd.to_numeric(df['fami_personashogar'], errors='coerce').astype('Int64')
print("Unique values in 'fami_personashogar' after cleaning:", df['fami_personashogar'].unique())

Unique values in 'fami_personashogar' after cleaning: <IntegerArray>
[3, 2, 5, 9, <NA>, 7, 6, 4, 8, 12, 10, 11]
Length: 12, dtype: Int64


#### 1.1.5 fami_tienecomputador

In [13]:
fami_tienecomputador = df['fami_tienecomputador'].unique()
print("Unique values in 'fami_tienecomputador':", fami_tienecomputador)

Unique values in 'fami_tienecomputador': ['Si' 'No' nan]


In [14]:
# Convert to binary 1/0
df['fami_tienecomputador'] = df['fami_tienecomputador'].replace({
    'Si': 1,
    'No': 0,
    'nan': 0,
}).astype('Int64')

print("Unique values in 'fami_tienecomputador' after cleaning:", df['fami_tienecomputador'].unique())

Unique values in 'fami_tienecomputador' after cleaning: <IntegerArray>
[1, 0, <NA>]
Length: 3, dtype: Int64


#### 1.1.6 fami_tieneinternet

In [15]:
fami_tieneinternet = df['fami_tieneinternet'].unique()
print("Unique values in 'fami_tieneinternet':", fami_tieneinternet)

Unique values in 'fami_tieneinternet': ['Si' 'No' nan]


In [16]:
# Convert to binary 1/0
df['fami_tieneinternet'] = df['fami_tieneinternet'].replace({
    'Si': 1,
    'No': 0,
    'nan': 0,
}).astype('Int64')  

print("Unique values in 'fami_tieneinternet' after cleaning:", df['fami_tieneinternet'].unique())

Unique values in 'fami_tieneinternet' after cleaning: <IntegerArray>
[1, 0, <NA>]
Length: 3, dtype: Int64


#### 1.1.7 fami_tieneautomovil

In [17]:
fami_tieneautomvil = df['fami_tieneautomovil'].unique()
print("Unique values in 'fami_tieneautomovil':", fami_tieneautomvil)

Unique values in 'fami_tieneautomovil': ['No' 'Si' nan]


In [18]:
# Convert to binary 1/0
df['fami_tieneautomovil'] = df['fami_tieneautomovil'].replace({
    'Si': 1,
    'No': 0,
    'nan': 0,
}).astype('Int64')  

print("Unique values in 'fami_tieneautomovil' after cleaning:", df['fami_tieneautomovil'].unique())

Unique values in 'fami_tieneautomovil' after cleaning: <IntegerArray>
[0, 1, <NA>]
Length: 3, dtype: Int64


#### 1.1.8 fami_tienelavadora

In [19]:
fami_tienelavadora = df['fami_tienelavadora'].unique()
print("Unique values in 'fami_tienelavadora':", fami_tienelavadora)

Unique values in 'fami_tienelavadora': ['Si' 'No' nan]


In [20]:
# Convert to binary 1/0
df['fami_tienelavadora'] = df['fami_tienelavadora'].replace({
    'Si': 1,
    'No': 0,
    'nan': 0,
}).astype('Int64')  

print("Unique values in 'fami_tienelavadora' after cleaning:", df['fami_tienelavadora'].unique())

Unique values in 'fami_tienelavadora' after cleaning: <IntegerArray>
[1, 0, <NA>]
Length: 3, dtype: Int64


#### 1.1.9 cole_naturaleza

In [21]:
cole_naturaleza = df['cole_naturaleza'].unique()
print("Unique values in 'cole_naturaleza':", cole_naturaleza)

Unique values in 'cole_naturaleza': ['OFICIAL' 'NO OFICIAL']


In [22]:
df['cole_naturaleza'] = df['cole_naturaleza'].replace({
    'OFICIAL': 'Público',
    'NO OFICIAL': 'Privado',
    'nan': np.nan
})

print("Unique values in 'cole_naturaleza' after cleaning:", df['cole_naturaleza'].unique())


Unique values in 'cole_naturaleza' after cleaning: ['Público' 'Privado']


#### 1.1.10 cole_area_ubicacion

In [23]:
cole_area_ubicacion = df['cole_area_ubicacion'].unique()
print("Unique values in 'cole_area_ubicacion':", cole_area_ubicacion)

Unique values in 'cole_area_ubicacion': ['URBANO' 'RURAL']


#### 1.1.11 cole_bilingue

In [24]:
cole_bilingue = df['cole_bilingue'].unique()
print("Unique values in 'cole_bilingue':", cole_bilingue)

Unique values in 'cole_bilingue': ['N' nan 'S']


In [25]:
df['cole_bilingue'] = df['cole_bilingue'].replace({
    'S': 1,
    'N': 0,
    'nan': 0,
}).astype('Int64')

print("Unique values in 'cole_bilingue' after cleaning:", df['cole_bilingue'].unique())

Unique values in 'cole_bilingue' after cleaning: <IntegerArray>
[0, <NA>, 1]
Length: 3, dtype: Int64


#### 1.1.12 estu_genero

In [26]:
estu_genero = df['estu_genero'].unique()
print("Unique values in 'estu_genero':", estu_genero)

Unique values in 'estu_genero': ['M' 'F' nan]


#### 1.1.13 Replace all nan values with np.nan in the dataframe

In [27]:
df = df.replace('nan', np.nan)